In [1]:
# Import necessary libraries
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch


from torch import nn
from data import TimeFusionDataset
from timefusion import TimeFusion
from utils.metrics import variogram_score, crps_sum
from utils.modules import EarlyStopper

In [2]:
# Import dataset
train_data = pd.read_csv("../datasets/electricity/train.csv").set_index("date")
test_data = pd.read_csv("../datasets/electricity/test.csv").set_index("date")
train_data = train_data.iloc[:,:20]
test_data = test_data.iloc[:,:20]

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
elif torch.backends.mps.is_available():
    device = torch.device("cpu")
    #device = torch.device("mps")
else:
    device = torch.device("cpu")

In [4]:
context_length = 30
prediction_length = 30

# Create each dataset
train_dataset = TimeFusionDataset(
    data = train_data.iloc[:int(0.9*len(train_data))],
    context_length = context_length,
)

val_dataset = TimeFusionDataset(
    data = train_data.iloc[int(0.9*len(train_data)):],
    context_length = context_length,
)

test_dataset = TimeFusionDataset(
    data = test_data,
    context_length = context_length,
)

train_loader = DataLoader(
    dataset = train_dataset,
    shuffle = True,
    num_workers = 4,
    batch_size = 128,
    #pin_memory=True,
    #pin_memory_device="cuda:0"
)

val_loader = DataLoader(
    dataset = val_dataset,
    shuffle = True,
    num_workers = 4,
    batch_size = 128,
    #pin_memory=True,
    #pin_memory_device="cuda:0"
)



In [5]:
predictor = TimeFusion(
    input_size = 20,
    hidden_size = 60,
    recurrent_layers = 2,
    dropout = 0.0,
    residual_layers = 2,
    scaling = True,
    device = device
)

print("Number of trainable parameters:",sum(p.numel() for p in predictor.parameters()))

Number of trainable parameters: 81324


/Users/edvard/Documents/Imperial College/Year 4/Final Year Project/Final-Year-Project/timefusion/diffusion.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.bar_alphas = torch.tensor(


In [6]:
optimizer = torch.optim.Adam(params=predictor.parameters(),lr=1e-3)
lr_scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=1, end_factor=0.01, total_iters=40)

predictor.train_network(
    train_loader = train_loader,
    epochs=40,
    val_loader = val_loader,
    val_metrics= {
        "Val MAE": nn.L1Loss(),
    },
    optimizer = optimizer,
    lr_scheduler= lr_scheduler,
    early_stopper=EarlyStopper(patience=10)
)

 |==============================|  Batch: 170 / 170, Epoch: 1 / 40, Average Loss: 0.7800


In [ ]:
# Load weights
predictor.load_state_dict(torch.load("../training_scripts/weights/2023-04-17-23-01-43",map_location=torch.device('cpu')))
predictor.eval()
#predictor.load_state_dict(torch.load("../weights/2023-04-12-20-02-45",map_location=torch.device('cpu')).state_dict())

FileNotFoundError: [Errno 2] No such file or directory: '../training_scripts/weights/2023-04-17-23-01-43'

In [ ]:
sample_index = 100

timestamps = []
for col in range(test_dataset.indices.shape[1]):
    timestamps.append(list(test_data.iloc[sample_index:,col].dropna().index[:predictor.prediction_length]))

timestamps = np.array(timestamps)

samples = predictor.sample(
    data = test_dataset,
    sample_indices = timestamps,
    num_samples = 100,
    timestamp_encodings = encodings
)

In [ ]:
realisations = []
for col in range(test_dataset.indices.shape[1]):
    realisations.append(test_dataset.data_copy.iloc[sample_index:,col].dropna()[:test_dataset.prediction_length])
realisations = np.array(realisations)

predictions = np.array(samples.cpu())

# Calculate metrics
var_score = variogram_score(realisations,predictions,**{"weights":"local","window_size":5})
crps_score = crps_sum(realisations,predictions)

print(var_score,crps_score)

nan nan


In [ ]:
# Plot the samples
confidence = 0.1

samples_cpu = samples.cpu().copy()
samples_cpu[:,0,:], _= torch.sort(samples_cpu[:,0,:],dim=0)
samples_cpu[:,1,:], _ = torch.sort(samples_cpu[:,1,:],dim=0)

plt.figure()
plt.plot(test_data.iloc[:sample_index,1].dropna().iloc[-predictor.context_length:],"-x")
plt.plot(test_data.iloc[sample_index:,1].dropna().iloc[:predictor.prediction_length],"-x")
plt.fill_between(timestamps[1], samples_cpu[int(confidence*samples_cpu.shape[0]),1,:], samples_cpu[int((1-confidence)*samples_cpu.shape[0]),1,:],alpha=0.5)
plt.plot(timestamps[1],torch.mean(samples.cpu(),axis=0)[1],"-o")

# plt.figure()
# plt.plot(val_data.data_copy.iloc[:sample_index,1].dropna().iloc[-predictor.context_length:],"-x")
# plt.plot(val_data.data_copy.iloc[sample_index:,1].dropna().iloc[:predictor.prediction_length],"-x")
# plt.fill_between(timestamps[1], samples_cpu[int(confidence*samples_cpu.shape[0]),1,:], samples_cpu[int((1-confidence)*samples_cpu.shape[0]),1,:],alpha=0.5)
# plt.plot(timestamps[1],torch.mean(samples.cpu(),axis=0)[1],"-o")


AttributeError: 'Tensor' object has no attribute 'copy'